# Data processing with Pandas, part 2

This week we will continue developing our skills using [Pandas](https://pandas.pydata.org/) to process real data. 

## Motivation

![Finland April 2019](img/Finland-April-2019.png)
*Source: [https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa)*

April 2019 was the [second warmest April on record globally](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa), and the warmest on record at 13 weather stations in Finland. 
In this lesson, we will use our data manipulation and analysis skills to analyze weather data from Finland, and investigate the claim that April 2019 was the warmest on record across Finland.

Along the way we will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task for several input files 

## Input data

In the lesson this week we are using weather observation data from Finland [downloaded from NOAA](https://www7.ncdc.noaa.gov/CDO/cdopoemain.cmd?datasetabbv=DS3505&countryabbv=&georegionabbv=&resolution=40). You will be working with data from either 15 or 4 different weather observation stations from Finland, depending on your environment. 

## Downloading the data

The first step for today's lesson is to get the data. Which data files you download will depend on the platform you're using for working through the lesson today. We recommend using the commandline tool [wget](https://www.gnu.org/software/wget/) for downloading the data. Wget is readily installed in the cloud computing environmets. 

### CSC Notebooks users

First, you need to open a new terminal window in Jupyter Lab (from **File** -> **New** -> **Terminal**). Once the terminal window is open, you will need to navigate to the L6 directory:

```bash
cd notebooks/L6/
```

You can confirm that you are located in the correct directory by listing the contents of the current directory:

```bash
ls
```

You should see something like the following output:

```bash
advanced-data-processing-with-pandas.ipynb errors.ipynb                               img
debugging.ipynb                            gcp-assertions.ipynb
```

If so, you're in the correct directory and you can download the data using wget:

```bash
wget https://davewhipp.github.io/data/Finland-weather-data-CSC.tar.gz
```

After the download completes, you can extract the data files using tar:

```bash
tar zxvf Finland-weather-data-CSC.tar.gz
```

At this stage you should have a new directory called `data` that contains the input data for this week's lesson. You can confirm this by listing the contents of the data-folder:

```bash
ls data
```

You should see something like the following:

```bash
029440.txt           029720.txt           3505doc.txt          6367598020644stn.txt
029700.txt           029740.txt           6367598020644inv.txt
```

Now you should be all set to proceed with the lesson!

### Users with Jupyter on their personal computers

If working on your own computer, you need to pay attention to the filepaths. First, you need to open a new terminal window in Jupyter Lab (from **File** -> **New** -> **Terminal**). Once the terminal window is open, you will need to navigate to the L6 directory:

```bash
cd path/to/L6/
```

where `path/to/` should be replaced with the correct path for the Lesson 6 materials on your computer. Once in the correct directory, you can confirm this by typing:

```bash
ls
```

You should see something like the following output:

```bash
advanced-data-processing-with-pandas.ipynb errors.ipynb                               img
debugging.ipynb                            gcp-assertions.ipynb
```

Next, you can download the data files using wget:

```bash
wget https://davewhipp.github.io/data/Finland-weather-data-full.tar.gz
```

After the download completes, you can extract the data files usign tar:

```bash
tar zxvf Finland-weather-data-full.tar.gz
```

At this stage you should have a new directory called `data` that contains the data for this week's lesson. You can confirm this by listing the contents of the data-folder:

```bash
ls data
```

You should see something like the following:

```bash
028360.txt           029070.txt           029440.txt           029740.txt  6367598020644inv.txt
028690.txt           029110.txt           029500.txt           029810.txt  6367598020644stn.txt
028750.txt           029170.txt           029700.txt           029820.txt
028970.txt           029350.txt           029720.txt           3505doc.txt
```

Now you should be all set to proceed with the lesson!

### Binder users

It is not recommended to complete this lesson using Binder.

## About the data

As part of the download there are a number of files that describe the weather data. These *metadata* files include:

- A list of stations\*: [data/6367598020644stn.txt](metadata/6367598020644stn.txt)
- Details about weather observations at each station: [data/6367598020644inv.txt](metadata/6367598020644inv.txt)
- A data description (i.e., column names): [data/3505doc.txt](metadata/3505doc.txt)

\*Note that the list of stations is for all 15 stations, even if you're working with only the 4 stations on the CSC Notebooks platform.

The input data for this week are separated with varying number of spaces (i.e., fixed width). The first lines and columns of the data look like following:

``` 
  USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP   ALT    STP MAX MIN PCP01 PCP06 PCP24 PCPXX SD
029440 99999 190601010600 090   7 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1011.0 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601011300 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1015.5 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601012000 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   25 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601020600 ***   0 *** *** CLR * * *  0.0 ** ** ** ** ** ** ** ** *   26 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** **
```

We will develop our analysis workflow using data for a single station. Then, we will repeat the same process for all the stations.

## Reading the data

In order to get started, let's import pandas: 

In [1]:
import pandas as pd

At this point, you can already have a quick look at the input file `029440.txt` for Tampere Pirkkala and how it is structured. We can notice at least two things we need to consider when reading in the data:

<div class="alert alert-info">

**Input data structure**

- **Delimiter:** The data are **separated with varying amount of spaces**. If you check out the documentation for the [read_csv() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), you can see that there are two different ways of doing this. We can use either `sep='\s+'` or `delim_whitespace=True` (but not both at the same time). In this case, we prefer to use `delim_whitespace` parameter.

- **No Data values:** No data values in the NOAA data are coded with varying number of `*`. We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.
</div>

In [2]:
# Define relative path to the file
fp = r"data/029440.txt"

# Read data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, na_values=['*', '**', '***', '****', '*****', '******'])

- Let's see how the data looks by printing the first five rows with the `head()` function

In [3]:
data.head()

,USAF,WBAN,YR--MODAHRMN,DIR,SPD,GUS,CLG,SKC,L,M,...,SLP,ALT,STP,MAX,MIN,PCP01,PCP06,PCP24,PCPXX,SD
0,29440,99999,190601010600,90.0,7.0,NaN,NaN,OVC,NaN,NaN,...,1011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29440,99999,190601011300,NaN,0.0,NaN,NaN,OVC,NaN,NaN,...,1015.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29440,99999,190601012000,NaN,0.0,NaN,NaN,OVC,NaN,NaN,...,1016.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29440,99999,190601020600,NaN,0.0,NaN,NaN,CLR,NaN,NaN,...,1016.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29440,99999,190601021300,270.0,7.0,NaN,NaN,OVC,NaN,NaN,...,1015.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All seems ok. However, we won't be needing all of the 33 columns for detecting warm temperatures in April. We can check all column names by running `data.columns`:

In [4]:
data.columns

Index(['USAF', 'WBAN', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'CLG', 'SKC', 'L',
       'M', 'H', 'VSB', 'MW', 'MW.1', 'MW.2', 'MW.3', 'AW', 'AW.1', 'AW.2',
       'AW.3', 'W', 'TEMP', 'DEWP', 'SLP', 'ALT', 'STP', 'MAX', 'MIN', 'PCP01',
       'PCP06', 'PCP24', 'PCPXX', 'SD'],
      dtype='object')

A description for all these columns is available in the metadata file [data/3505doc.txt](metadata/3505doc.txt). 

**Let's read in the data one more time.** This time, we will read in only some of the columns using the `usecols` parameter. Let's read in columns that might be somehow useful to our analysis, or at least that contain some values that are meaningful to us, including the station name, timestamp, and data about wind and temperature: `'USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`

In [5]:
# Read in only selected columns
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

# Check the dataframe
data.head()

,USAF,YR--MODAHRMN,DIR,SPD,GUS,TEMP,MAX,MIN
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN


Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterisk (\*) characters into `NaN` values. 

## Renaming columns

As we saw above some of the column names are a bit awkward and difficult to interpret. Luckily, it is easy to alter labels in a pandas DataFrame using the [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)-function. In order to change the column names, we need to tell pandas how we want to rename the columns using a dictionary that lists old and new column names

Let's first check again the current column names in our DataFrame:

In [6]:
data.columns

Index(['USAF', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'TEMP', 'MAX', 'MIN'], dtype='object')

<div class="alert alert-info">

**Dictionaries**

[Dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) is a spesific data structure in Python for storing key-value pairs. During this course, we will use dictionaries mainly when renaming columns in a pandas series, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).
    
</div>



We can define the new column names using a [dictionary](https://www.tutorialspoint.com/python/python_dictionary.htm) where we determine "`key: value`" -pairs, in which the original column name (the one which will be replaced) is the key, and the new column name is the value.

- Let's change:
   
   - `YR--MODAHRMN` column into `TIME`, 
   - `SPD` into `SPEED`, and
   - `GUS` into `GUST`

In [7]:
# Create the dictionary with old and new names
new_names = {'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}

# Let's see what the variable new_names look like
new_names

{'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}

In [8]:
# Check the data type of the new_names variable
type(new_names)

dict

From above we can see that we have successfully created a new dictionary. 

- Now we can change the column names by passing that dictionary into parameter `columns` in `rename()` -function.

In [9]:
# Rename the columns
data = data.rename(columns=new_names)

# Print the new columns
print(data.columns)

Index(['USAF', 'TIME', 'DIR', 'SPEED', 'GUST', 'TEMP', 'MAX', 'MIN'], dtype='object')


Perfect, now our column names are more easy to understand and use. 

<div class="alert alert-info">

**TASK: Renaming columns**

The temperature values are again in Fahrenheit. As you might guess, we will soon convert these temperatures in to Celsius. In order to avoid confusion with the columns, rename column `TEMP` into `TEMP_F`. Let's also rename `USAF` as`STATION_NUMBER`.
</div>


In [10]:
# Create the dictionary with old and new names
new_names = {'USAF':'STATION_NUMBER', 'TEMP': 'TEMP_F'}

# Rename the columns
data = data.rename(columns=new_names)

# Check the output
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN


## Check dataframe properties

As we learned last week, it's always a good idea to check basic properties of the input data before proceeding with data analysis. Let's check:

- How many rows and columns we have:

In [11]:
data.shape

(74940, 8)

- Top and bottom rows: 

In [12]:
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN


In [13]:
data.tail()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
74935,29440,201910012220,130.0,3.0,NaN,39.0,NaN,NaN
74936,29440,201910012250,110.0,3.0,NaN,37.0,NaN,NaN
74937,29440,201910012300,100.0,2.0,NaN,38.0,NaN,NaN
74938,29440,201910012320,100.0,3.0,NaN,37.0,NaN,NaN
74939,29440,201910012350,110.0,3.0,NaN,37.0,NaN,NaN


- Data types of the columns: 

In [14]:
data.dtypes

STATION_NUMBER      int64
TIME                int64
DIR               float64
SPEED             float64
GUST              float64
TEMP_F            float64
MAX               float64
MIN               float64
dtype: object

- Descriptive statistics:

In [15]:
print(data.describe())

       STATION_NUMBER          TIME           DIR         SPEED          GUST  \
count         74940.0  7.494000e+04  71794.000000  74488.000000  10271.000000   
mean          29440.0  2.011518e+11    306.184222      7.143755     16.706747   
std               0.0  2.590599e+09    294.547944      4.665385      5.186162   
min           29440.0  1.906010e+11     10.000000      0.000000     11.000000   
25%           29440.0  2.017072e+11    140.000000      5.000000     13.000000   
50%           29440.0  2.018041e+11    210.000000      7.000000     15.000000   
75%           29440.0  2.019011e+11    320.000000      9.000000     19.000000   
max           29440.0  2.019100e+11    990.000000     59.000000     43.000000   

             TEMP_F          MAX          MIN  
count  74933.000000  1900.000000  1901.000000  
mean      42.348018    46.622105    37.624934  
std       17.382721    17.735917    16.878257  
min      -22.000000    -2.000000   -16.000000  
25%       30.000000    33.0000

Here we can see that there are varying number of observations per column (see the `count` information), because some of the columns have missing values.

## Using your own functions in Pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply self-made functions to data in a pandas DataFrame.
**In short, our task is to define a function for the temperature conversion, and to apply this function for each Celsius value on each row of the DataFrame. Output celsius values should be stored in a new column called** `TEMP_C`.

Knowing how to use your own function in pandas can be really useful when doing your own analyses. Here, we will introduce two different approaches for using function in pandas. First, we will see how we can apply the function row-by-row using a `for`-loop and the `DataFrame.iterrows()`-method, and then we will learn how to apply the method to all rows at once using [DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

For both of these approaches, we first need to define our temperature conversion function from Fahrenheit to Celsius:

In [16]:
def fahr_to_celsius(temp_fahrenheit):
    """
    Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius
    converted_temp = (temp_fahrenheit - 32) / 1.8
    
    return converted_temp

With such a simple example, we can use the function direcly on a column in the DataFrame in order to conver the values:

In [17]:
data["TEMP_C"] = fahr_to_celsius(data["TEMP_F"])

data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778


If we want to do something more complicated, we need to know how to apply the function row-by-row.

### Iterating over rows

We can iterate over the rows of Pandas DataFrame by using the `iterrows()` -method and use the function one row at a time.


When iterating over the rows in our `DataFrame`, it is noteworthy to understand that pandas actually keeps track on the `index` value as well. Hence, the contents of a single row actually contains not only the values, but also the `index` of that row (each row is a pandas Series!). 

- Let's see how `iterrows()` works by printing out the `TEMP` value on each row using a `for`-loop:


In [18]:
# Iterate over the rows
for idx, row in data.iterrows():
    # Print the index value
    print('Index:', idx)
    
    # Print the row
    print('Temp F:', row["TEMP_F"], "\n")
    
    break

Index: 0
Temp F: 27.0 



<div class="alert alert-info">

**Breaking a loop**

When developing a for-loop, you don't always need to go trough the whole loop if you just want to test things out. 
[Break](https://www.tutorialspoint.com/python/python_break_statement.htm) statement in Python terminates the current loop after the first iteration and we used it here just to test check out the values on the first row.
With a large data, you might not want to print out thousands of values to the screen!
    
</div>


We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

- Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values into that column using the `fahr_to_celsius` function we defined earlier:


In [19]:
# Create an empty column for the DataFrame where the values will be stored
new_column = "TEMP_C"
data[new_column] = None

# Iterate over the rows 
for idx, row in data.iterrows():
    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row['TEMP_F'])
    
    # Update the value of 'Celsius' column with the converted value
    data.at[idx, new_column] = celsius

<div class="alert alert-info">

**Reminder: .at or .loc?**

Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result. 
    
If you only need to access a single value in a DataFrame, [DataFrame.at](https://pandas.pydata.org/pandas-docs/version/0.25/reference/api/pandas.DataFrame.at.html) is faster compared to [DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) which is designed for accessing groups of rows and columns. 
    
</div>

- Let's see what we have now.

In [20]:
data.head(10)

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.77778
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.77778
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.88889
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.33333
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.77778
5,29440,190601022000,NaN,0.0,NaN,27.0,NaN,NaN,-2.77778
6,29440,190601030600,270.0,7.0,NaN,26.0,NaN,NaN,-3.33333
7,29440,190601031300,270.0,7.0,NaN,25.0,NaN,NaN,-3.88889
8,29440,190601032000,270.0,7.0,NaN,24.0,NaN,NaN,-4.44444
9,29440,190601040600,NaN,0.0,NaN,18.0,NaN,NaN,-7.77778


Great! Now we have converted our temperatures into Celsius by using our self-made function.

### Applying a function

Pandas DataFrames and Series also have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parenthesis!) as an argument:

In [21]:
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778


**Note:** pay attention which column you are applying the function on! Running this code: `data.apply(fahr_to_celsius)` would not give an error, but the results also don't make much sense.

<div class="alert alert-info">

**Should I use .iterrows() or .apply()?**

We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping trough DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 
    
    
At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!
    
</div>

## Parsing dates

We will eventually want to group our data based on month in order to see if April temperatures in 2019 were higher than average. Currently, the date and time information is stored in the column `TIME`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type:

In [22]:
data["TIME"].head(10)

0    190601010600
1    190601011300
2    190601012000
3    190601020600
4    190601021300
5    190601022000
6    190601030600
7    190601031300
8    190601032000
9    190601040600
Name: TIME, dtype: int64

In [23]:
data["TIME"].tail(10)

74930    201910012050
74931    201910012100
74932    201910012120
74933    201910012150
74934    201910012200
74935    201910012220
74936    201910012250
74937    201910012300
74938    201910012320
74939    201910012350
Name: TIME, dtype: int64

The `TIME` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `190601010600`, i.e. from 1st of January 1906 (way back!), and the timestamp for the latest observation is `201910012350` (from last week, by the time of writing this!)

In [24]:
data["TIME"].dtypes

dtype('int64')

And the information is stored as integer values.

There are several different options for proceeding from here. The bottom line is, that we would want to **aggregate the data on a monthly level**, and in order to do so we need to "label" each row of data based on the month when the record was observed. 
In practice, we could create a new column (or an index), which contains information about the month (including the year, but excluding days, hours and minutes).

### String slicing

One approach would be to convert the date and time information into character strings and "cut" the needed information from the [string objects](https://docs.python.org/3/tutorial/introduction.html#strings). If we look at the latest time stamp in the data (`201910012350`), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and six first characters are year + month!

In [25]:
date = "201910012350"
date[0:6]

'201910'

Doing this in pandas requires two steps:
  1. Convert the `TIME` column from `int` into `str` datatype.
  2. Slice the correct range of characters from the character string using [pandas.Series.str.slice()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html)

- Let's convert the time into string. And check that the data type changes:

In [26]:
# Convert to string
data['TIME_STR'] = data['TIME'].astype(str)

In [27]:
# SLice the string
data['YEAR_MONTH'] = data['TIME_STR'].str.slice(start=0, stop=6)

# Let's see what we have
print(data.head())

   STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29440  190601010600   90.0    7.0   NaN    27.0  NaN  NaN   
1           29440  190601011300    NaN    0.0   NaN    27.0  NaN  NaN   
2           29440  190601012000    NaN    0.0   NaN    25.0  NaN  NaN   
3           29440  190601020600    NaN    0.0   NaN    26.0  NaN  NaN   
4           29440  190601021300  270.0    7.0   NaN    27.0  NaN  NaN   

     TEMP_C      TIME_STR YEAR_MONTH  
0 -2.777778  190601010600     190601  
1 -2.777778  190601011300     190601  
2 -3.888889  190601012000     190601  
3 -3.333333  190601020600     190601  
4 -2.777778  190601021300     190601  


Nice! Now we have "labeled" the rows based on information about day of the year and hour of the day. However, let's have a look at a more clever way of dealing with dates and times..

### datetime

<div class="alert alert-info">

**Pandas datetime**

In pandas, we can convert dates and times into a new data type [datetime](https://docs.python.org/3.7/library/datetime.html) using [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function. First, it is important to understand the structure of the input data in order to avoid erroneous conversions, and that's why we first learned string slicing before introducing the datetime functionalities. 
    
Here is one example of how to convert the `TIME_STR`-column in our data set to datetime:
    
```
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])

```
   
        
If needed, you can use the `format` parameter to define the output datetime format according to [strftime(format) method](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior). together with `exact=False`, for example like this: 
    
```
# Convert to datetime
data["YEAR_MONTH"] = pd.to_datetime(data["TIME_STR"], format='%Y%m', exact=False)

```
In this example, `exact=False` drops out days, hours and minutes, because they are not included in the specified formatting.
 
</div>




In [28]:
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])
data["DATE"].head()

0   1906-01-01 06:00:00
1   1906-01-01 13:00:00
2   1906-01-01 20:00:00
3   1906-01-02 06:00:00
4   1906-01-02 13:00:00
Name: DATE, dtype: datetime64[ns]

Note: in this case, the data type of the values is `datetime`. 

<div class="alert alert-info">

**Pandas Series datetime properties**
    
There are several methods available for accessing information about the properties of datetime values. Read more from the pandas documentation about [datetime properties](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties).
</div>

Now, we can extract different time units based on the datetime-column using the [pandas.Series.dt accessor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html):

In [29]:
data['DATE'].dt.year

0        1906
1        1906
2        1906
3        1906
4        1906
         ... 
74935    2019
74936    2019
74937    2019
74938    2019
74939    2019
Name: DATE, Length: 74940, dtype: int64

We can also combine the datetime functionalities with other methods from pandas. For example, we can check the number of unique years in our input data: 

In [30]:
data['DATE'].dt.year.nunique()

7

<div class="alert alert-info">

**TASK:**

- Create two new columns: `YEAR` and `MONTH` based on the date column
</div>

In [31]:
data['YEAR'] = data['DATE'].dt.year
data['MONTH'] = data['DATE'].dt.month

data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,TIME_STR,YEAR_MONTH,DATE,YEAR,MONTH
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601010600,190601,1906-01-01 06:00:00,1906,1
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778,190601011300,190601,1906-01-01 13:00:00,1906,1
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889,190601012000,190601,1906-01-01 20:00:00,1906,1
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333,190601020600,190601,1906-01-02 06:00:00,1906,1
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601021300,190601,1906-01-02 13:00:00,1906,1


## Aggregating data in Pandas by grouping

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) which is a handy method for compressing large amounts of data and computing statistics for subgroups.

Our practical task is to calculate the average temperatures for each month

This can be done by aggregating the data, i.e.:

  1. **grouping the data** based on year and month
  2. Calculating the average for each month (each group) either by using a for-loop or directly from the grouped object
  3. Storing those values into **a new DataFrame** `monthly_data`

Before we start grouping the data, let's once more check how our input data looks like:

In [32]:
print("number of rows:", len(data))
data.head()

number of rows: 74940


,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,TIME_STR,YEAR_MONTH,DATE,YEAR,MONTH
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601010600,190601,1906-01-01 06:00:00,1906,1
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778,190601011300,190601,1906-01-01 13:00:00,1906,1
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889,190601012000,190601,1906-01-01 20:00:00,1906,1
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333,190601020600,190601,1906-01-02 06:00:00,1906,1
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601021300,190601,1906-01-02 13:00:00,1906,1


We have quite a few rows of weather data, and several observations per day. **Our goal is to create an aggreated data frame that would have only one row per month!**

Let's **group** our data based on unique year and month combinations

In [33]:
grouped = data.groupby(["YEAR", "MONTH"])

<div class="alert alert-info">

**NOTE:**
    
Here you could also group the data based on the `YEAR_MONTH` column to achieve the same result:
    
```
# Group the data 
grouped = data.groupby('YEAR_MONTH')
```
</div>

- Let's see what we have now.

In [34]:
# What is the type?
print("Type:\n", type(grouped))

# How many?
print("Length:\n", len(grouped))

Type:
 <class 'pandas.core.groupby.generic.DataFrameGroupBy'>
Length:
 82


Okey, interesting. Now we have a new object with type `DataFrameGroupBy`.

<div class="alert alert-info">

**TASK:**

Think: what does the number of groups (length of the grouped object) tell us?
</div>

In [35]:
# Answer: the length of the grouped object should be the same as
data["YEAR_MONTH"].nunique()

# in other words, the number of groups is the number of unique year and month combinations in our data

82

There are several methods we can use for extracting information from the grouped data. See [documentation for Pandas GroupBy objects](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) for a comprehensive overview. 

**Checking group names:**

In [36]:
# Check the "names" of each group (uncomment the next row if you want to print out all the keys)
#grouped.groups.keys()

**Accessing data for one group:**

- Let's check the contents for a group representing August 2019 (name of that group is `(2019, 4)` if you grouped the data based on datetime columns `YEAR` and `MONTH`). We can get the values of that hour from the grouped object using the `get_group()` -method:

In [37]:
# Specify the time of the first hour (as text)
month = (2019, 4)

# Select the group
group1 = grouped.get_group(month)

# Let's see what we have
print(group1)

       STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
62055           29440  201904010000  280.0    6.0   NaN    30.0  NaN  NaN   
62056           29440  201904010020  280.0    7.0   NaN    32.0  NaN  NaN   
62057           29440  201904010050  280.0    6.0   NaN    30.0  NaN  NaN   
62058           29440  201904010100  280.0    7.0   NaN    30.0  NaN  NaN   
62059           29440  201904010120  280.0    6.0   NaN    30.0  NaN  NaN   
...               ...           ...    ...    ...   ...     ...  ...  ...   
64213           29440  201904302220  990.0    1.0   NaN    36.0  NaN  NaN   
64214           29440  201904302250  990.0    1.0   NaN    36.0  NaN  NaN   
64215           29440  201904302300  360.0    0.0   NaN    36.0  NaN  NaN   
64216           29440  201904302320  990.0    1.0   NaN    34.0  NaN  NaN   
64217           29440  201904302350  190.0    3.0   NaN    36.0  NaN  NaN   

         TEMP_C      TIME_STR YEAR_MONTH                DATE  YEAR  MONTH  

Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific month. Let's check the DataType of this group:

In [38]:
type(group1)

pandas.core.frame.DataFrame

So, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics etc for this spesific group. 
We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` -function that we already used during the Lesson 5. 

- Let's calculate the mean for following attributes all at once:
   - `DIR`, 
   - `SPEED`, 
   - `GUST`, 
   - `TEMP`, 
   - `TEMP_C`
   - `MONTH` 

In [39]:
# Specify the columns that will be part of the calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', 'MONTH']

# Calculate the mean values all at one go
mean_values = group1[mean_cols].mean()

# Let's see what we have
print(mean_values)

DIR       309.035306
SPEED       5.932188
GUST       15.868217
TEMP_F     42.472030
TEMP_C      5.817794
MONTH       4.000000
dtype: float64


Here we saw how you can access data from a single group. For getting information about all groups (all months) we can a `for` -loop or methods available in the grouped object.

**For-loops and grouped objects:**

When iterating over the groups in our `DataFrameGroupBy` -object
it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening).


In [40]:
# Iterate over groups
for key, group in grouped:
    # Print key and group
    print("Key:\n", key)
    print("\nFirst rows of data in this group:\n", group.head())
    
    # Stop iteration with break command
    break

Key:
 (1906, 1)

First rows of data in this group:
    STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29440  190601010600   90.0    7.0   NaN    27.0  NaN  NaN   
1           29440  190601011300    NaN    0.0   NaN    27.0  NaN  NaN   
2           29440  190601012000    NaN    0.0   NaN    25.0  NaN  NaN   
3           29440  190601020600    NaN    0.0   NaN    26.0  NaN  NaN   
4           29440  190601021300  270.0    7.0   NaN    27.0  NaN  NaN   

     TEMP_C      TIME_STR YEAR_MONTH                DATE  YEAR  MONTH  
0 -2.777778  190601010600     190601 1906-01-01 06:00:00  1906      1  
1 -2.777778  190601011300     190601 1906-01-01 13:00:00  1906      1  
2 -3.888889  190601012000     190601 1906-01-01 20:00:00  1906      1  
3 -3.333333  190601020600     190601 1906-01-02 06:00:00  1906      1  
4 -2.777778  190601021300     190601 1906-01-02 13:00:00  1906      1  


Okey so from here we can see that the `key` contains the name of the group (year, month).

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeat slightly the earlier steps so that you can see and better understand what is happening.

In [41]:
# Create an empty DataFrame for the aggregated values
monthly_data = pd.DataFrame()

# The columns that we want to aggregate
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', "MONTH"]

# Iterate over the groups
for key, group in grouped:
    
   # Calculate mean
   mean_values = group[mean_cols].mean()

   # Add the ´key´ (i.e. the date+time information) into the aggregated values
   mean_values['YEAR_MONTH'] = key

   # Append the aggregated values into the DataFrame
   monthly_data = monthly_data.append(mean_values, ignore_index=True)

- Let's see what we have now:

In [42]:
print(monthly_data)

           DIR       GUST  MONTH      SPEED     TEMP_C     TEMP_F  YEAR_MONTH
0   218.181818        NaN    1.0  13.204301  -3.596177  25.526882   (1906, 1)
1   178.095238        NaN    2.0  13.142857  -3.445767  25.797619   (1906, 2)
2   232.043011        NaN    3.0  15.021505  -5.107527  22.806452   (1906, 3)
3   232.045455        NaN    4.0  13.811111   3.790123  38.822222   (1906, 4)
4   192.820513        NaN    5.0  10.333333  13.070490  55.526882   (1906, 5)
..         ...        ...    ...        ...        ...        ...         ...
77  370.992008  17.251852    6.0   8.138490  16.524111  61.743400   (2019, 6)
78  294.433641  15.034722    7.0   5.785714  16.427753  61.569955   (2019, 7)
79  320.335766  15.751678    8.0   6.769447  15.888138  60.598649   (2019, 8)
80  306.491058  15.173285    9.0   6.363594   9.976743  49.958137   (2019, 9)
81  239.577465  17.470588   10.0  10.169014   5.985915  42.774648  (2019, 10)

[82 rows x 7 columns]


Awesome! Now we have aggregated our data and we have a new DataFrame called `monthly_data` where we have mean values for each month in the data set.

**Mean for all groups at once**

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object:

In [43]:
grouped.mean()

STATION_NUMBER          TIME         DIR      SPEED       GUST  \
YEAR MONTH                                                                   
1906 1             29440.0  1.906012e+11  218.181818  13.204301        NaN   
     2             29440.0  1.906021e+11  178.095238  13.142857        NaN   
     3             29440.0  1.906032e+11  232.043011  15.021505        NaN   
     4             29440.0  1.906042e+11  232.045455  13.811111        NaN   
     5             29440.0  1.906052e+11  192.820513  10.333333        NaN   
...                    ...           ...         ...        ...        ...   
2019 6             29440.0  2.019062e+11  370.992008   8.138490  17.251852   
     7             29440.0  2.019072e+11  294.433641   5.785714  15.034722   
     8             29440.0  2.019082e+11  320.335766   6.769447  15.751678   
     9             29440.0  2.019092e+11  306.491058   6.363594  15.173285   
     10            29440.0  2.019100e+11  239.577465  10.169014  17.470588   

               TEMP_F        MAX        MIN     TEMP_C  
YEAR MONTH                                              
1906 1      25.526882        NaN        NaN  -3.596177  
     2      25.797619        NaN        NaN  -3.445767  
     3      22.806452        NaN        NaN  -5.107527  
     4      38.822222        NaN        NaN   3.790123  
     5      55.526882        NaN        NaN  13.070490  
...               ...        ...        ...        ...  
2019 6      61.743400  67.316667  55.600000  16.524111  
     7      61.569955  67.774194  55.903226  16.427753  
     8      60.598649  65.935484  55.016129  15.888138  
     9      49.958137  53.766667  45.350000   9.976743  
     10     42.774648  48.500000  41.000000   5.985915  

[82 rows x 9 columns]

## Detecting warm months

Now, we have aggregated our data on monthly level and all we need to do is to check which years had the warmest April temperatures. A simple approach is to select all aprils from the data, group the data and check which group(s) have the highest mean value:

- select all records that are from April (regardless of the year):

In [44]:
aprils = data[data["MONTH"]==4]

- take a subset of columns that might contain interesting information:

In [45]:
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C','YEAR', 'MONTH']]

- group by year and month:

In [46]:
grouped = aprils.groupby(by=["YEAR", "MONTH"])

- calculate mean for each group:

In [47]:
monthly_mean = grouped.mean()
monthly_mean.head()

,,STATION_NUMBER,TEMP_F,TEMP_C
YEAR,MONTH,,,
1906,4,29440,38.822222,3.790123
1907,4,29440,36.111111,2.283951
1908,4,29440,36.811111,2.672840
1909,4,29440,31.977778,-0.012346
2017,4,29440,34.766620,1.537011


- check the highest temperature values (sort the data frame in a descending order):

In [48]:
monthly_mean.sort_values(by="TEMP_C", ascending=False).head(10)

,,STATION_NUMBER,TEMP_F,TEMP_C
YEAR,MONTH,,,
2019,4,29440,42.472030,5.817794
2018,4,29440,38.951887,3.862159
1906,4,29440,38.822222,3.790123
1908,4,29440,36.811111,2.672840
1907,4,29440,36.111111,2.283951
2017,4,29440,34.766620,1.537011
1909,4,29440,31.977778,-0.012346


How did April 2019 rank at the Tampere Pirkkala observation station 🌡️? 

## Repeating the data analysis with larger dataset


Finally, let's repeat the data analysis steps above for all the available data we have (!!). First, confirm the path to the **folder** where all the input data are located. 
The idea is, that we will repeat the analysis process for each input file using a (rather long) for loop! Here we have all the main analysis steps with some additional output info - all in one long code cell:

In [49]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

# Rename the columns
new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
data = data.rename(columns=new_names)

#Print info about the current input file:
print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
print("NUMBER OF OBSERVATIONS:", len(data))

# Create column
col_name = 'TEMP_C'
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

# Convert TIME to string 
data['TIME_STR'] = data['TIME'].astype(str)

# Parse year and month
data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April 
aprils = data[data['MONTH']==4]

# Take a subset of columns
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

# Group by year and month
grouped = aprils.groupby(by=["YEAR", "MONTH"])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
print("\n")

STATION NUMBER: 29440
NUMBER OF OBSERVATIONS: 74940
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2019 4               29440  42.472030  5.817794
2018 4               29440  38.951887  3.862159
1906 4               29440  38.822222  3.790123
1908 4               29440  36.811111  2.672840
1907 4               29440  36.111111  2.283951




We will use the `glob()` function from the module `glob` to list our input files. 

In [50]:
import glob

In [51]:
file_list = glob.glob(r'data/0*txt')

<div class="alert alert-info">

**Note**

Note that we're using the \* character as a wildcard, so any file that starts with `data/0` and ends with `txt` will be added to the list of files we will iterate over. We specifically use `data/0` as the starting part of the file names to avoid having our metadata files included in the list!
</div>

In [52]:
print("Number of files in the list", len(file_list))
print(file_list)

Number of files in the list 1
['data\\029440.txt']


Now, you should have all the relevant file names in a list, and we can loop over the list using a for-loop:

In [53]:
for fp in file_list:
    print(fp)

data\029440.txt


In [54]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

    # Rename the columns
    new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
    data = data.rename(columns=new_names)

    #Print info about the current input file:
    print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
    print("NUMBER OF OBSERVATIONS:", len(data))

    # Create column
    col_name = 'TEMP_C'
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

    # Convert TIME to string 
    data['TIME_STR'] = data['TIME'].astype(str)

    # Parse year and month
    data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
    data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April 
    aprils = data[data['MONTH']==4]

    # Take a subset of columns
    aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

    # Group by year and month
    grouped = aprils.groupby(by=["YEAR", "MONTH"])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
    print("\n")

STATION NUMBER: 29440
NUMBER OF OBSERVATIONS: 74940
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2019 4               29440  42.472030  5.817794
2018 4               29440  38.951887  3.862159
1906 4               29440  38.822222  3.790123
1908 4               29440  36.811111  2.672840
1907 4               29440  36.111111  2.283951




How about now, how did April 2019 rank across different stations?